In [2]:
import numpy as np
from keras import layers
from keras.layers import Input, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D
from keras.layers import AveragePooling2D, MaxPooling2D, Dropout, GlobalMaxPooling2D, GlobalAveragePooling2D
from keras.utils import np_utils
from keras.models import Sequential
from keras.callbacks import ModelCheckpoint
import pandas as pd

import keras.backend as K

def keras_model(image_x, image_y):
    num_of_classes = 3
    model = Sequential()
    model.add(Conv2D(32, (5, 5), input_shape=(image_x, image_y, 1), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='same'))
    model.add(Conv2D(64, (5, 5), activation='sigmoid'))
    model.add(MaxPooling2D(pool_size=(5, 5), strides=(5, 5), padding='same'))
    model.add(Flatten())
    model.add(Dense(1024, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(num_of_classes, activation='softmax'))

    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    filepath = "emojinator.h5"
    checkpoint1 = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
    callbacks_list = [checkpoint1]

    return model, callbacks_list

def main():
    data = pd.read_csv("train_foo.csv")
    dataset = np.array(data)
    np.random.shuffle(dataset)
    X = dataset
    Y = dataset
    X = X[:, 1:2501]
    Y = Y[:, 0]

    X_train = X[0:1200, :]
    X_train = X_train / 255.
    X_test = X[30:60, :]
    X_test = X_test / 255.

    # Reshape
    Y = Y.reshape(Y.shape[0], 1)
    Y_train = Y[0:1200, :]
    Y_train = Y_train.T
    Y_test = Y[30:60, :]
    Y_test = Y_test.T

    print("number of training examples = " + str(X_train.shape[0]))
    print("number of test examples = " + str(X_test.shape[0]))
    print("X_train shape: " + str(X_train.shape))
    print("Y_train shape: " + str(Y_train.shape))
    print("X_test shape: " + str(X_test.shape))
    print("Y_test shape: " + str(Y_test.shape))
    image_x = 50
    image_y = 50

    train_y = np_utils.to_categorical(Y_train)
    test_y = np_utils.to_categorical(Y_test)
    train_y = train_y.reshape(train_y.shape[1], train_y.shape[2])
    test_y = test_y.reshape(test_y.shape[1], test_y.shape[2])
    X_train = X_train.reshape(X_train.shape[0], 50, 50, 1)
    X_test = X_test.reshape(X_test.shape[0], 50, 50, 1)
    print("X_train shape: " + str(X_train.shape))
    print("X_test shape: " + str(X_test.shape))

    model, callbacks_list = keras_model(image_x, image_y)
    model.fit(X_train, train_y, validation_data=(X_test, test_y), epochs=10, batch_size=64,
              callbacks=callbacks_list)
    scores = model.evaluate(X_test, test_y, verbose=0)
    print("CNN Error: %.2f%%" % (100 - scores[1] * 100))

    model.save('emojinator.h5')


main()

number of training examples = 199
number of test examples = 30
X_train shape: (199, 2500)
Y_train shape: (1, 199)
X_test shape: (30, 2500)
Y_test shape: (1, 30)
X_train shape: (199, 50, 50, 1)
X_test shape: (30, 50, 50, 1)
Train on 199 samples, validate on 30 samples
Epoch 1/10
199/199 [==============================] - 2s 8ms/step - loss: 2.1456 - accuracy: 0.4724 - val_loss: 2.6486 - val_accuracy: 0.5000
Epoch 2/10


C:\Users\KIIT\Anaconda3\lib\site-packages\keras\callbacks\callbacks.py:707: RuntimeWarning: Can save best model only with val_acc available, skipping.
  'skipping.' % (self.monitor), RuntimeWarning)


199/199 [==============================] - 1s 3ms/step - loss: 2.2187 - accuracy: 0.5025 - val_loss: 0.9382 - val_accuracy: 0.5000
Epoch 3/10
199/199 [==============================] - 1s 3ms/step - loss: 1.3328 - accuracy: 0.4975 - val_loss: 0.8386 - val_accuracy: 0.5000
Epoch 4/10
199/199 [==============================] - 1s 4ms/step - loss: 0.7168 - accuracy: 0.6080 - val_loss: 1.0098 - val_accuracy: 0.5000
Epoch 5/10
199/199 [==============================] - 1s 3ms/step - loss: 0.8558 - accuracy: 0.5075 - val_loss: 0.4202 - val_accuracy: 1.0000
Epoch 6/10
199/199 [==============================] - 1s 3ms/step - loss: 0.3907 - accuracy: 0.9799 - val_loss: 0.2746 - val_accuracy: 1.0000
Epoch 7/10
199/199 [==============================] - 1s 3ms/step - loss: 0.2584 - accuracy: 1.0000 - val_loss: 0.1478 - val_accuracy: 1.0000
Epoch 8/10
199/199 [==============================] - 1s 4ms/step - loss: 0.1194 - accuracy: 1.0000 - val_loss: 0.0606 - val_accuracy: 1.0000
Epoch 9/10
199/19